# Cone Detection using EfficientDet-0

#### Connecting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

### [TensorFlow-models](https://github.com/tensorflow/models) Installation  

In [ ]:
!pip install tensorflow==2.7.0

#### Cloning the repository

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
      os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

#### Python package installation

In [ ]:
%cd /content/drive/MyDrive/models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .  
!python -m pip install .

#### Testing the installation

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

#### Generating TFrecords
> TensorFlow uses tfrecords(a simple format for storing a sequence of binary records) to train an machine learning model.<br>
> Using ```generate_tfrecord.py```, training and validation records are generated.

In [ ]:
%cd /content/drive/MyDrive/workspace/eff_det
# Create test data:
!python generate_tfrecord.py -x test_images/ -l annotations/label_map.pbtxt -o annotations/test.record

# Create train data:
!python generate_tfrecord.py -x images/ -l annotations/label_map.pbtxt -o annotations/train.record

#### Uisng TensorBoard for monitoring the results
> The training logs are stored in ./output_training. With the help of TensorBaord, visualisation will be done.<br> 
> Before, starting to train the model, I initliase the tensorboard so that the graphs are visible during training.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/drive/MyDrive/workspace/eff_det/models/efficientdet_d0_coco17_tpu-32/

#### Start Training process
> Before starting to train the model, check the config pipeline.
> In case you face an error with OpenCV execute code block below, else skip it



In [ ]:
# In case of an error, ensure OpenCV version is set
!pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.5.2.52

In [ ]:
%cd /content/drive/MyDrive/workspace/eff_det/
#train 
!python model_main_tf2.py --model_dir=models/efficientdet_d0_coco17_tpu-32 --pipeline_config_path=models/efficientdet_d0_coco17_tpu-32/pipeline.config


#### Creating Inference Graph
> Exporting the inference graph. Inference graph is the saved model which we have trained. Now I will use these for testing the model.

In [ ]:
%cd /content/drive/MyDrive/workspace/eff_det/

!python exporter_main_v2.py --input_type image_tensor \
--pipeline_config_path models/efficientdet_d0_coco17_tpu-32/pipeline.config \
--trained_checkpoint_dir models/efficientdet_d0_coco17_tpu-32/ \
--output_directory final_model/

#### Inference on image

In [ ]:
"""
Object Detection (On Image) From tf2.0 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
from google.colab.patches import cv2_imshow

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = '/content/drive/MyDrive/workspace/eff_det/test_images/753_jpg.rf.5a1b80aa587b9f51d1ceddc886074cad.jpg'

# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = '/content/drive/MyDrive/workspace/eff_det/final_model/saved_model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = '/content/drive/MyDrive/workspace/eff_det/annotations/label_map.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image_ = cv2.imread(IMAGE_PATHS)
image = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)
# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.5,
      agnostic_mode=False)

print('Done')
# DISPLAYS OUTPUT IMAGE
image_with_detections = cv2.cvtColor(image_with_detections, cv2.COLOR_BGR2RGB)
cv2_imshow(image_with_detections)
# CLOSES WINDOW ONCE KEY IS PRESSED
